In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# LDDMM landmark dynamics

In [ ]:
from src.manifolds.landmarks import *
M = landmarks(2)
print(M)

from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

In [ ]:
# example configuration
M.k_sigma.set_value(tensor(np.diag((.5,.5))))

q = tensor(np.vstack((np.linspace(-.5,.5,M.N.eval()),np.zeros(M.N.eval()))).T.flatten())
v = tensor(np.vstack((np.zeros(M.N.eval()),np.ones(M.N.eval()))).T.flatten())
p = M.flatf(q,v)
print("q = ", q)
print("p = ", p)

## Geodesics

In [ ]:
# 2nd order geodesic equation
from src.Riemannian import geodesic
geodesic.initialize(M)

qs = M.Exptf(q,v)
M.plot()
M.plotx(qs,v,linewidth=1.5)
plt.show()

In [ ]:
# Hamiltonian dynamics
print(M.Hf(q,p))
from src.dynamics import Hamiltonian
Hamiltonian.initialize(M)

# geodesic
qs = M.Exp_Hamiltoniantf(q,p).T
M.plot()
M.plotx(qs,v)
plt.show()
(ts,qps) = M.Hamiltonian_dynamicsf(q,p)
ps = qps[:,1,:]
print("Energy: ",np.array([M.Hf(q,p) for (q,p) in zip(qs,ps)]))

# Boundary value problem

In [ ]:
# Logarithm map
from src.Riemannian import Log
Log.initialize(M,f=M.Exp_Hamiltonian)

p_Log = M.Logf(q,qs[-1])[0]
print(p_Log)
print(p)

qs = M.Exp_Hamiltoniantf(q,p_Log).T
M.plot()
M.plotx(qs,v)
plt.show()

## Curvature

In [ ]:
from src.Riemannian import curvature
curvature.initialize(M)
print("curvature shape= ", M.Rf(q).shape)
# print("curvature = ", M.Rf(q))
# Ricci and scalar curvature:
print("Ricci curvature = ", M.Ricci_curvf(q))
print("Scalar curvature = ", M.S_curvf(q))

In [ ]:
# plot min of Ricci curvature tensor between two landmarks, one fixed at x1=(0,0)
x1 = tensor(np.array([0.,0.]))

# grids
pts = 40 # even number to avoid (0,0), high value implies nicer plot but extended computation time
border = .2
minx = -border
maxx = +border
miny = -border
maxy = +border
X, Y = np.meshgrid(np.linspace(minx,maxx,pts),np.linspace(miny,maxy,pts))
xy = np.vstack([X.ravel(), Y.ravel()]).T        

# plot
newfig()
cmap = cm.jet
alpha = 1
ax = plt.gca()
fs = np.array([np.min(np.real(np.linalg.eigvals(
    np.dot(M.gsharpf(tensor(np.concatenate((x1,x)))),
           M.Ricci_curvf(tensor(np.concatenate((x1,x))))
          )))) for x in xy])
norm = mpl.colors.Normalize(vmin=np.min(fs),vmax=np.max(fs))
colors = cmap(norm(fs)).reshape(X.shape+(4,))
surf = ax.plot_surface(X, Y, fs.reshape(X.shape), rstride=1, cstride=1, cmap=cmap, facecolors = colors,  linewidth=0., antialiased=True, alpha=alpha, edgecolor=(0,0,0,0), shade=False)
m = cm.ScalarMappable(cmap=surf.cmap,norm=norm)
m.set_array(colors)
plt.colorbar(m, shrink=0.7)
ax.set_xlim3d(minx,maxx), ax.set_ylim3d(miny,maxy), ax.set_zlim3d(np.min(fs)-1,np.max(fs)+1)

## Brownian Motion

In [ ]:
# coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

(ts,qs) = M.Brownian_coordsf(q,dWsf(M.dim.eval()))
M.plot()
M.plotx(qs)
plt.show()

## Frame Bundle Geodesics

In [ ]:
from src.framebundle import FM
FM.initialize(M)

In [ ]:
# landmarks
x = tensor(np.array([[0,1],[0.5,1]]).flatten())
# frame
nu = tensor(np.array([[1,0],[0,1],[1,0],[0,1]]))

u = np.concatenate((x,nu.flatten()))

px = tensor(np.array([0,1,0,1]))
pnu = np.zeros_like(nu)
p = np.concatenate((px,pnu.flatten()))
print("nu = ", nu)
print("u = ", u)
print("p = ", p)

In [ ]:
# Hamiltionian
print(M.H_FMf(u,p))

# compute FM geodesic
us = M.Exp_Hamiltonian_FMtf(u,p).T

M.plotFMx(us,N_vec=5)
plt.show()
(ts,qps) = M.Hamiltonian_dynamics_FMf(u,p)
ps = qps[:,1,:]
print("Energy: ",np.array([M.H_FMf(q,p) for (q,p) in zip(us,ps)]))